In [3]:
# To install `pandas_datareader`: Already installed
'''
pip install pandas_datareader
'''

'\npip install pandas_datareader\n'

In [ ]:
# Making the same request repeatedly can use a lot of bandwidth, slow down your code and may result in your IP being banned. `pandas-datareader` allows you to cache queries into a `sqlite` database using `requests_cache`. First make sure you have installed `requests_cache`:

'''
pip install requests_cache
'''

In [12]:
# The following code pulls BMW historical (split and dividend) adjusted price between 1/1/2001 and 1/2/2021 from yahoo finance:
import pandas as pd 
import pandas_datareader as pdr
df_bmw = pdr.get_data_yahoo("BMW.DE", start="2001-01-01", end="2021-02-01")
pd.concat([df_bmw.head(5),df_bmw.tail(5)],axis=0)  #看看收尾的5行

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2001-01-01,34.900002,34.900002,34.900002,34.900002,0.0,20.443398
2001-01-02,36.200001,34.150002,35.730000,34.470001,420631.0,20.191515
2001-01-03,35.799999,32.400002,34.220001,34.529999,1593510.0,20.226660
2001-01-04,36.040001,33.799999,35.150002,35.110001,1203012.0,20.566406
2001-01-05,35.110001,35.110001,35.110001,35.110001,0.0,20.566406
2021-01-26,73.209999,69.779999,70.160004,72.080002,2034716.0,72.080002
2021-01-27,72.029999,70.500000,71.940002,71.309998,2044649.0,71.309998
2021-01-28,71.080002,68.360001,69.279999,70.910004,2072544.0,70.910004
2021-01-29,70.870003,69.050003,69.400002,70.010002,1924221.0,70.010002


In [13]:
# By specifying `adjust_price=True`, `pandas_datareader` adjusts all prices (`Open`, `High`, `Low`, `Close`) based on `Adj Close` price. It also adds `Adj_Ratio` column and drops `Adj Close`.

df_bmw_adj = pdr.get_data_yahoo("BMW.DE", start="2001-01-01", end="2021-02-01", adjust_price=True)
df_bmw_adj.head()

,High,Low,Open,Close,Volume,Adj_Ratio
Date,,,,,,
2001-01-01,20.443398,20.443398,20.443398,20.443398,0.0,0.585771
2001-01-02,21.204898,20.004069,20.929585,20.191515,420631.0,0.585771
2001-01-03,20.970589,18.978970,20.045072,20.226660,1593510.0,0.585771
2001-01-04,21.111173,19.799046,20.589838,20.566406,1203012.0,0.585771
2001-01-05,20.566406,20.566406,20.566406,20.566406,0.0,0.585771


In [18]:
# You may also download multiple stock data at the same time, though you need some pandas kung-fu to transform the shape of the dataframe:

data = pdr.get_data_yahoo(["ADS.DE", "BMW.DE"], start="2001-01-01", end="2021-02-01", adjust_price=True)
df_multi = data.stack(level=1).reset_index(level=1)
df_multi.head(10)

Attributes,Symbols,Adj_Ratio,Close,High,Low,Open,Volume
Date,,,,,,,
2001-01-01,ADS.DE,0.749004,12.358573,12.358573,12.358573,12.358573,0.0
2001-01-01,BMW.DE,0.585771,20.443398,20.443398,20.443398,20.443398,0.0
2001-01-02,ADS.DE,0.749004,12.171323,12.208772,11.916661,11.974709,1607584.0
2001-01-02,BMW.DE,0.585771,20.191515,21.204898,20.004069,20.929585,420631.0
2001-01-03,ADS.DE,0.749004,11.909169,11.944747,11.609568,11.892317,506560.0
2001-01-03,BMW.DE,0.585771,20.226660,20.970589,18.978970,20.045072,1593510.0
2001-01-04,ADS.DE,0.749004,12.358573,12.639450,11.796820,12.077696,894416.0
2001-01-04,BMW.DE,0.585771,20.566406,21.111173,19.799046,20.589838,1203012.0
2001-01-05,ADS.DE,0.749004,12.358573,12.358573,12.358573,12.358573,0.0


In [22]:
# The following function can download and cache your requests for BMW historical price between 1/1/2001 and today from yahoo finance. `web.DataReader()` exposes limited parameters, therefore we can only get adjusted closing price:

from datetime import timedelta, date
import pandas_datareader.data as web
import requests_cache

def get_data(symbols, start="1/1/2001", end=date.today()):  ##### 用这个来获取data！！！！！！！！！！
    session = requests_cache.CachedSession(
        cache_name="cache", backend="sqlite", expire_after=timedelta(days=1))
    data = web.DataReader(symbols, "yahoo", start, end, session=session)
    return data

get_data("BMW.DE").head(10)


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2001-01-01,34.900002,34.900002,34.900002,34.900002,0.0,20.443398
2001-01-02,36.200001,34.150002,35.730000,34.470001,420631.0,20.191515
2001-01-03,35.799999,32.400002,34.220001,34.529999,1593510.0,20.226660
2001-01-04,36.040001,33.799999,35.150002,35.110001,1203012.0,20.566406
2001-01-05,35.110001,35.110001,35.110001,35.110001,0.0,20.566406
2001-01-08,37.299999,36.400002,37.000000,37.500000,1178138.0,21.966396
2001-01-09,37.000000,36.349998,37.000000,37.080002,1174091.0,21.720377
2001-01-10,36.650002,36.000000,36.500000,36.000000,785169.0,21.087746
2001-01-11,35.549999,34.049999,35.549999,34.849998,1313572.0,20.414104
